In [1]:

import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')

import dos


In [2]:

simulation_path = './dos/BendingModes'
sim = dos.DOS(simulation_path, show_timing = 2)


INFO:DOS:Reading config from ./dos/BendingModes/dos.yaml
INFO:DOS:New driver: wind_loads
INFO:wind_loads:New output: Gravity
INFO:wind_loads:New output: Distributed
INFO:wind_loads:New output: TopEnd
INFO:wind_loads:New output: Truss
INFO:wind_loads:New output: GIR
INFO:wind_loads:New output: CRING
INFO:wind_loads:New output: M1
INFO:wind_loads:New output: M1Cell
INFO:wind_loads:New output: M2
INFO:wind_loads:New output: M2Cell
INFO:DOS:New driver: FEM
INFO:FEM:New input: OSS_Grav_CS0
INFO:FEM:New input: OSS_M1_lcl_6F
INFO:FEM:New input: OSS_Cell_lcl_6F
INFO:FEM:New input: OSS_CRING_6F
INFO:FEM:New input: OSS_TopEnd_6F
INFO:FEM:New input: OSS_GIR_6F
INFO:FEM:New input: OSS_Truss_6F
INFO:FEM:New input: MC_M2_lcl_force_6F
INFO:FEM:New input: MC_M2_MacroCell_F_6d
INFO:FEM:New input: M1_distributed_windF
INFO:FEM:New output: Bending_modes
INFO:FEM:Output logged in!
INFO:Gravity:Linked to OSS_Grav_CS0 from FEM
INFO:Distributed:Linked to M1_distributed_windF from FEM
INFO:TopEnd:Linked to OS

SIMCEO server received: Acknowledging connection from SIMCEO client!


In [3]:

sim.start()


INFO:DOS:Pushing configuration to server
INFO:wind_loads:WindLoad
INFO:FEM:FEM
INFO:DOS:Initializing
INFO:wind_loads:None
INFO:FEM:The server has failed!
INFO:DOS:Running


In [ ]:

sim.pctComplete


### Read and plot LOS data

---


In [ ]:
import pickle
import numpy as np
import scipy.io as sio

file_path = "../CUDAIMS-NSEE/datafiles/CUDAIMSsysDt/memorymaps/dgnf.dat"
dgnf_mm = np.memmap(file_path, dtype=np.float32, mode='r+', shape=(21,84))
dgnf = dgnf_mm.view(dtype = np.float32, type = np.ndarray)

OSS_M1_LOS = sim.logs.entries['FEM']['OSS_M1_LOS'].timeSeries[1].T
MC_M2_LOS_6D = sim.logs.entries['FEM']['MC_M2_LOS_6D'].timeSeries[1].T

dec = 10
dt  = OSS_M1_LOS.shape[1]
nt  = dt // dec

LOS_outs = np.memmap('./LOS.dat', dtype=np.float32, mode='w+', shape=(21,dt))
for k in range(dec):
    ki, kii = nt * k, nt * (k + 1)
    if kii < dt:
        outs = np.concatenate((OSS_M1_LOS[:,ki:kii], MC_M2_LOS_6D[:,ki:kii]), axis=0)
    else:
        outs = np.concatenate((OSS_M1_LOS[:,ki:], MC_M2_LOS_6D[:,ki:]), axis=0)
    los  = dgnf @ outs 
    LOS_outs[:,ki:kii] = los[:,:]
    
LOS_sim = sio.loadmat('./LOS.mat')['outs']

In [ ]:
index = 5

signal = LOS_outs[index, :10000]
sim_signal = LOS_sim[index, :10000]

plt.figure(figsize = (26,18))
plt.plot(signal, lw = 3.0)
plt.plot(sim_signal, lw = 1.3)
plt.show()

### Plotting simulation logs

---

Specifically plot the bending modes outputs


In [ ]:
plt.figure(figsize = (16,8))
plt.plot(*sim.logs.entries['FEM']['Bending_modes'].timeSeries)
plt.show()

### Save a particular output in memory map

---

In [ ]:
signal_values.shape

In [ ]:
import numpy as np

signal_name   = 'bendingmodes_SS'
signal_values = sim.logs.entries['FEM']['Bending_modes'].timeSeries[1]

saved_mem_map = np.memmap('./variables/'+signal_name+'.dat', dtype=np.float32, mode='w+', shape=signal_values.shape)
saved_mem_map[:,:] = signal_values[:,:]
del saved_mem_map

### Infomation analysis of the bending modes outputs

---



In [ ]:
import numpy as np

shape__ = (20000, 1144)
bm_ssr = np.memmap('./variables/bendingmodes_SS.dat', dtype=np.float32, mode='r+', shape=shape__)
bm_stf = np.memmap('./variables/bendingmodes_ST.dat', dtype=np.float32, mode='r+', shape=shape__)

index = 15

plt.figure(figsize=(16,8))
plt.plot(bm_ssr[:,index], '-', label='Steady State Response')
plt.plot(bm_stf[:,index], '--', label='Stiffness Matrix')
plt.legend()
plt.show()


### Terminate the simulation

---

In [ ]:
sim.terminate

## Current Developments

---

### Bending mode validation

Here one wanna compare the displacements computed from the simulated bending modes, and the displacements simulated with FEM. Therefore, one might read the states saved and compute the displacements

In [ ]:

import numpy as np
import scipy.io as sio

from utils import LargeCompute


In [ ]:

shape_st_ = (11000,20000)
states = np.memmap('./variables/states.dat', dtype=np.float32, mode='r+', shape=shape_st_)

C = sio.loadmat('../database/matmodels/consolidated_withSSBM.mat')['C']


data = sio.loadmat('./dos/BendingModes/FEM_IO_WBM.mat')
fem_outputs=[(x[0][0],y[0]) for x,y in zip(data['FEM_IO']['outputs_name'][0,0],data['FEM_IO']['outputs_size'][0][0])]

i_sum = 0
for k in fem_outputs:
    if k[0] != 'M1_surfaces_d':
        if k[0] != 'Bending_modes':
            i_sum += k[1]
    else:
        ref_size = k[1]
i_, ii_ = i_sum, i_sum + ref_size


C = C[i_:ii_,:]

indexes = range(2,C.shape[0],3)
C = C[indexes, :]


lc = LargeCompute()
displacements_ptt = lc.dot_time(C, states)

In [ ]:

shape_bm_ = (20000, 1144)
bending_modes = np.memmap('./variables/bendingmodes_SS.dat', dtype=np.float32, mode='r+', shape=shape_bm_)

#shape_dp_ = (20000, 5127)
#displacements = np.memmap('./variables/displacements.dat', dtype=np.float64, mode='r+', shape=shape_dp_)

Read the transformation matrix

In [ ]:
import scipy.io as sio

U_ = sio.loadmat('./variables/auxiliar/SSBM_transformation_matrices.mat')['U_']

Q_ = sio.loadmat('./variables/auxiliar/SSBM_transformation_matrices.mat')['Q_']


Recover the displacements from bending modes:

In [ ]:
shape_dp_ = (5127, 20000)

lc = LargeCompute()

bm_displacements = np.memmap('./variables/bm_displacements_.dat', dtype=np.float32, mode='w+', shape=shape_dp_)
displacements = np.memmap('./variables/displacements_.dat', dtype=np.float32, mode='w+', shape=shape_dp_)
for k in range(U_.shape[0]):
    i = k * 165
    ki = k * 742
    if k != U_.shape[0]-1:
        ii = (k + 1) * 165 
        kii = (k + 1) * 742
    else:
        ii = i + 154
        kii = ki + 675
    bm_displacements[ki:kii,:] = lc.bm_to_disp_time(U_[k,0], bending_modes[:,i:ii].T)
    displacements[ki:kii,:] = lc.remove_ptt_from_disp(Q_[k,0], displacements_ptt[ki:kii,:])

print('Simulated displacements shape: ', displacements.shape)
print('Bending modes displacements shape: ', bm_displacements.shape)

In [ ]:
import scipy.linalg as sli

def remove_ptt(ptt, displacements, vartype=np.float32):

    #    get the shape of the displacements
    _size = displacements.shape
    result = np.zeros(_size)
    for k in range(_size[1]):
        _ptt = sli.lstsq(ptt, displacements[:,k])[0]
        print(displacements[:,k].shape, _ptt.shape, ptt.shape)
        result[:,k] = displacements[:,k] - np.dot(ptt, np.array(_ptt))

    return result

In [ ]:
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')

index = 1000

plt.figure(figsize=(16,8))
plt.plot(displacements_ptt[index,:])
plt.show()


In [ ]:
displacements_ptt.shape

In [ ]:
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style(context='talk', fscale=1.4, spines=False, gridlines='--')

index = 50

plt.figure(figsize=(16,16))
plt.subplot(3,1,1)
plt.plot(displacements[index,:], '-', c='C1', label='Simulated without PTT')
plt.legend()
plt.subplot(3,1,2)
plt.plot(bm_displacements[index,:], c='C2', label='From bending modes')
plt.legend()
plt.subplot(3,1,3)
plt.plot(displacements_ptt[index,:],'-', c='C3', label='Simulated with PTT')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,64))
plt.matshow(U_[0,0], 0)
plt.show()